In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np

# Set default renderer to browser to avoid nbformat dependency
# pio.renderers.default = "browser"

# Sample data
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
min_temp = [68, 65, 70, 72, 69, 73, 71]
max_temp = [82, 79, 84, 89, 85, 90, 86]

# Create a DataFrame for easier data manipulation
df = pd.DataFrame({
    'day': days,
    'min_temp': min_temp,
    'max_temp': max_temp
})

# Create figure
fig = go.Figure()

# Add filled area for temperature range
fig.add_trace(
    go.Scatter(
        x=df['day'],
        y=df['max_temp'],
        mode='lines',
        line=dict(width=0.5, color='rgb(255, 127, 14)'),
        name='Max Temperature'
    )
)
fig.add_trace(
    go.Scatter(
        x=df['day'],
        y=df['min_temp'],
        fill='tonexty',  # Fill to the y trace before this one
        mode='lines',
        line=dict(width=0.5, color='rgb(31, 119, 180)'),
        name='Min Temperature'
    )
)

# Update layout
fig.update_layout(
    title='Weekly Temperature Range',
    xaxis_title='Day of Week',
    yaxis_title='Temperature (°F)',
    hovermode='x unified'
)

# Show the figure - this will now open in your browser
fig.show()

In [ ]:
# Generate sample stock data
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', periods=30)
price = 100 + np.cumsum(np.random.normal(0, 1, 30))
ma_20 = pd.Series(price).rolling(window=7).mean()
volatility = pd.Series(price).rolling(window=5).std() * 2

# Create figure
fig = go.Figure()

# Add stock price
fig.add_trace(go.Scatter(
    x=dates,
    y=price,
    mode='lines',
    name='Stock Price',
    line=dict(color='darkgreen', width=2)
))

# Add moving average
fig.add_trace(go.Scatter(
    x=dates,
    y=ma_20,
    mode='lines',
    name='7-Day MA',
    line=dict(color='orange', width=2)
))

# Add upper volatility band
fig.add_trace(go.Scatter(
    x=dates,
    y=ma_20 + volatility,
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

# Add lower volatility band with fill
fig.add_trace(go.Scatter(
    x=dates,
    y=ma_20 - volatility,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',  # Fill between this trace and the previous one
    fillcolor='rgba(255, 165, 0, 0.2)',  # Light orange with transparency
    name='Volatility'
))

# Update layout
fig.update_layout(
    title='Stock Price with Moving Average and Volatility Bands',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    hovermode='x unified'
)

fig.show()



In [ ]:
# Sample data for energy production by source over years
years = list(range(2015, 2025))
solar = [5, 7, 12, 18, 24, 35, 45, 60, 75, 90]
wind = [20, 25, 30, 40, 45, 55, 65, 75, 85, 95]
hydro = [40, 42, 41, 43, 44, 45, 46, 47, 48, 50]
fossil = [100, 95, 90, 85, 80, 70, 60, 50, 40, 30]

# Create figure
fig = go.Figure()

# Add traces for each energy source
fig.add_trace(go.Scatter(
    x=years, y=solar,
    mode='lines',
    line=dict(width=0.5, color='rgb(255, 215, 0)'),
    stackgroup='one',
    name='Solar'
))

fig.add_trace(go.Scatter(
    x=years, y=wind,
    mode='lines',
    line=dict(width=0.5, color='rgb(100, 149, 237)'),
    stackgroup='one',
    name='Wind'
))

fig.add_trace(go.Scatter(
    x=years, y=hydro,
    mode='lines',
    line=dict(width=0.5, color='rgb(0, 191, 255)'),
    stackgroup='one',
    name='Hydro'
))

fig.add_trace(go.Scatter(
    x=years, y=fossil,
    mode='lines',
    line=dict(width=0.5, color='rgb(128, 128, 128)'),
    stackgroup='one',
    name='Fossil Fuels'
))

# Update layout
fig.update_layout(
    title='Energy Production by Source (2015-2024)',
    xaxis_title='Year',
    yaxis_title='Energy Production (TWh)',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

In [ ]:
# Generate sample data for website traffic
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
traffic = [1500, 1700, 2000, 2200, 2400, 2300, 2100, 2000, 2500, 2800, 3000, 3500]
baseline = [1000] * 12

# Create figure
fig = go.Figure()

# Add baseline
fig.add_trace(go.Scatter(
    x=months,
    y=baseline,
    mode='lines',
    line=dict(color='rgba(0,0,0,0)'),
    showlegend=False
))

# Add traffic data with gradient fill
fig.add_trace(go.Scatter(
    x=months,
    y=traffic,
    mode='lines+markers',
    line=dict(color='darkblue', width=2),
    marker=dict(size=8, color='darkblue'),
    fill='tonexty',
    fillgradient=dict(
        type="vertical",
        colorscale=[(0, "white"), (0.5, "lightblue"), (1, "royalblue")]
    ),
    name='Website Visitors'
))

# Update layout
fig.update_layout(
    title='Monthly Website Traffic (2024)',
    xaxis_title='Month',
    yaxis_title='Visitors',
    hovermode='x unified'
)

fig.show()


In [ ]:
# Generate sample data for revenue and costs
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
revenue = [250000, 310000, 275000, 350000]
costs = [180000, 200000, 190000, 210000]
profit = [r - c for r, c in zip(revenue, costs)]

# Create custom hover text
hover_text = [f"Quarter: {q}<br>Revenue: ${r:,}<br>Costs: ${c:,}<br>Profit: ${p:,}"
              for q, r, c, p in zip(quarters, revenue, costs, profit)]

# Create figure
fig = go.Figure()

# Add revenue line
fig.add_trace(go.Scatter(
    x=quarters,
    y=revenue,
    mode='lines+markers',
    name='Revenue',
    line=dict(color='green', width=2),
    marker=dict(size=10),
    hoverinfo='text',
    hovertext=hover_text
))

# Add costs line with fill to represent profit
fig.add_trace(go.Scatter(
    x=quarters,
    y=costs,
    mode='lines+markers',
    name='Costs',
    line=dict(color='red', width=2),
    marker=dict(size=10),
    fill='tonexty',  # Fill area between traces
    fillcolor='rgba(144, 238, 144, 0.5)',  # Light green with transparency
    hoverinfo='text',
    hovertext=hover_text
))

# Update layout
fig.update_layout(
    title='Quarterly Financial Performance',
    xaxis_title='Quarter',
    yaxis_title='Amount ($)',
    hovermode='x unified'
)

fig.show()
